# Libraries

In [1]:
#libraries
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import re
import matplotlib.pyplot as plt
import os
import numpy as np
sign_index=1


# Read and Scale Image

In [2]:
def read_and_scale_image(path):
    image = cv2.imread(path)
    if path !="C:\\Users\\Nutech\\Desktop\\Semester 7\\Gen AI\\test code\\genai\\img\\images\\image16.jpg":
        image = cv2.resize(image, (888, 1212))
    scale_percent = 50  
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)

    resize=cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
    return resize

# Extract Signatures

In [3]:
def extract_rows_and_save_images(image, output_directory='output_images'):
    # Define dimensions of each row and columns
    box_height = 50
    column_widths = [36, 105, 105, 105, 105]  # widths for each of the 5 columns

    # Calculate the number of rows in the image
    num_rows = image.shape[0] // box_height

    # Ensure output directory exists
    os.makedirs(output_directory, exist_ok=True)

    for row in range(num_rows):
        # Calculate the y-coordinate for the current row
        y_start = row * box_height
        y_end = y_start + box_height
        
        # Crop the row image
        row_image = image[y_start:y_end, :]

        # Extract the ID from the first column
        id_region = row_image[:, :column_widths[0]]  # First column
        id_gray = cv2.cvtColor(id_region, cv2.COLOR_BGR2GRAY)
        _, id_thresh = cv2.threshold(id_gray, 200, 255, cv2.THRESH_BINARY_INV)

        # Find contours to extract the ID number
        id_contours, _ = cv2.findContours(id_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if id_contours:
            id_contour = max(id_contours, key=cv2.contourArea)
            x, y, w, h = cv2.boundingRect(id_contour)
            id_number = id_region[y:y+h, x:x+w]
            id_number = cv2.resize(id_number, None, fx=4, fy=4, interpolation=cv2.INTER_CUBIC)

            global sign_index

            id_folder = os.path.join(output_directory,str(sign_index))
            sign_index+=1
            os.makedirs(id_folder, exist_ok=True)



            # Loop through the sign columns (2nd to 5th)
            for col in range(1, 5):  # 1 to 4 for sign columns
                sign_x_start = sum(column_widths[:col])  # Start x-coordinate for the sign
                sign_x_end = sign_x_start + column_widths[col]  # End x-coordinate for the sign

                # Crop the sign image
                sign_image = row_image[:, sign_x_start:sign_x_end]

                # Save the sign image in the respective folder
                sign_image_path = os.path.join(id_folder, f'sign_{col}.png')
                cv2.imwrite(sign_image_path, sign_image)

            print(f"Saved images for ID {sign_index} in folder: {id_folder}")


In [4]:
global sign_index
sign_index=1

# Process Images

In [5]:
import os
import cv2
# Function to sort filenames numerically
def sort_key(filename):
    numbers = re.findall(r'\d+', filename)
    return int(numbers[0]) if numbers else filename

# Function to process images in the directory
def process_images_in_directory(directory_path, output_directory='output_images'):
    # Ensure the output directory exists
    os.makedirs(output_directory, exist_ok=True)

    # List image files and sort them numerically
    image_filenames = [f for f in os.listdir(directory_path) if f.endswith(('.png', '.jpg'))]
    sorted_filenames = sorted(image_filenames, key=sort_key)

    # Loop over all sorted image files
    for filename in sorted_filenames:
        image_path = os.path.join(directory_path, filename)

        # Process the image (e.g., extract rows and save images)
        image = read_and_scale_image(image_path)
        if image is not None:
            extract_rows_and_save_images(image, output_directory)

# Example usage
process_images_in_directory('C:\\Users\\Nutech\\Desktop\\Semester 7\\Gen AI\\test code\\genai\\img\\images', 'output_images')

Saved images for ID 2 in folder: output_images\1
Saved images for ID 3 in folder: output_images\2
Saved images for ID 4 in folder: output_images\3
Saved images for ID 5 in folder: output_images\4
Saved images for ID 6 in folder: output_images\5
Saved images for ID 7 in folder: output_images\6
Saved images for ID 8 in folder: output_images\7
Saved images for ID 9 in folder: output_images\8
Saved images for ID 10 in folder: output_images\9
Saved images for ID 11 in folder: output_images\10
Saved images for ID 12 in folder: output_images\11
Saved images for ID 13 in folder: output_images\12
Saved images for ID 14 in folder: output_images\13
Saved images for ID 15 in folder: output_images\14
Saved images for ID 16 in folder: output_images\15
Saved images for ID 17 in folder: output_images\16
Saved images for ID 18 in folder: output_images\17
Saved images for ID 19 in folder: output_images\18
Saved images for ID 20 in folder: output_images\19
Saved images for ID 21 in folder: output_images\

# Augment Images

In [6]:
import os
import cv2
import numpy as np
import imgaug.augmenters as iaa

# Function to apply augmentations to an image
def augment_image(image):
    augmenters = iaa.Sequential([
        iaa.Fliplr(0.5),  # 50% chance to apply horizontal flip
        iaa.Flipud(0.5),  # 50% chance to apply vertical flip
        iaa.Affine(rotate=(-45, 45)),  # Rotate between -45 and 45 degrees
        iaa.Affine(scale=(0.8, 1.2)),  # Scale the image
        iaa.Multiply((0.8, 1.2)),  # Change brightness
        iaa.AdditiveGaussianNoise(scale=(0, 0.05*255))  # Add Gaussian noise
    ])
    return augmenters(image=image)

# Function to process a folder of images
def augment_images_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)

        # Check if the file is an image
        if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.jpeg'):
            # Read the image
            image = cv2.imread(img_path)
            # Apply augmentation
            augmented_images = [augment_image(image) for _ in range(10)]  # Create 10 augmented versions
            
            # Save the augmented images in the same folder
            for i, aug_img in enumerate(augmented_images):
                aug_filename = f"aug_{i}_{filename}"
                aug_img_path = os.path.join(folder_path, aug_filename)
                cv2.imwrite(aug_img_path, aug_img)
                print(f"Saved {aug_img_path}")

# Main function to process all folders
def augment_all_images_in_output_directory(output_dir):
    for folder_name in os.listdir(output_dir):
        folder_path = os.path.join(output_dir, folder_name)

        # Check if the path is a directory
        if os.path.isdir(folder_path):
            print(f"Processing folder: {folder_path}")
            augment_images_in_folder(folder_path)

# Set the path to the 'output images' directory
output_directory = 'output_images'

# Augment images in all subfolders
augment_all_images_in_output_directory(output_directory)


Processing folder: output_images\1
Saved output_images\1\aug_0_sign_1.png
Saved output_images\1\aug_1_sign_1.png
Saved output_images\1\aug_2_sign_1.png
Saved output_images\1\aug_3_sign_1.png
Saved output_images\1\aug_4_sign_1.png
Saved output_images\1\aug_5_sign_1.png
Saved output_images\1\aug_6_sign_1.png
Saved output_images\1\aug_7_sign_1.png
Saved output_images\1\aug_8_sign_1.png
Saved output_images\1\aug_9_sign_1.png
Saved output_images\1\aug_0_sign_2.png
Saved output_images\1\aug_1_sign_2.png
Saved output_images\1\aug_2_sign_2.png
Saved output_images\1\aug_3_sign_2.png
Saved output_images\1\aug_4_sign_2.png
Saved output_images\1\aug_5_sign_2.png
Saved output_images\1\aug_6_sign_2.png
Saved output_images\1\aug_7_sign_2.png
Saved output_images\1\aug_8_sign_2.png
Saved output_images\1\aug_9_sign_2.png
Saved output_images\1\aug_0_sign_3.png
Saved output_images\1\aug_1_sign_3.png
Saved output_images\1\aug_2_sign_3.png
Saved output_images\1\aug_3_sign_3.png
Saved output_images\1\aug_4_s

# Model Training

In [7]:
import os
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Define the image dimensions
IMG_HEIGHT, IMG_WIDTH = 128, 128
BATCH_SIZE = 32
EPOCHS = 20

# Function to load and preprocess images
def load_images_from_folders(base_path):
    images = []
    labels = []
    
    # Loop over each folder (unique ID)
    for folder_name in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder_name)
        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                img_path = os.path.join(folder_path, filename)
                if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.jpeg'):
                    # Read and resize the image
                    image = cv2.imread(img_path)
                    image = cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH))
                    images.append(image)
                    labels.append(folder_name)  # Use folder name as the label (ID)
    
    return np.array(images), np.array(labels)

# Function to preprocess the data and split into training and validation sets
def preprocess_data(base_path):
    # Load images and labels
    images, labels = load_images_from_folders(base_path)
    
    # Normalize the image data
    images = images.astype('float32') / 255.0

    # Encode labels to integers
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels)
    num_classes = len(np.unique(encoded_labels))
    
    # One-hot encode the labels
    labels = to_categorical(encoded_labels, num_classes)
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
    
    return X_train, X_test, y_train, y_test, num_classes, label_encoder

# Function to build the CNN model
def build_cnn_model(num_classes, IMG_HEIGHT=128, IMG_WIDTH=128):
    model = Sequential([
        # First Convolutional Layer
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        #MaxPooling2D(pool_size=(2, 2)),
        
        # Second Convolutional Layer
        Conv2D(64, (3, 3), activation='relu'),
        #MaxPooling2D(pool_size=(2, 2)),
        
        # Third Convolutional Layer
        Conv2D(128, (3, 3), activation='relu'),
        #MaxPooling2D(pool_size=(2, 2)),

        # Third Convolutional Layer
        Conv2D(256, (3, 3), activation='relu'),
        #MaxPooling2D(pool_size=(2, 2)),

        # Third Convolutional Layer
        Conv2D(512, (3, 3), activation='relu'),
        #MaxPooling2D(pool_size=(2, 2)),

        # Flattening
        Flatten(),
        
        # Fully Connected Layer
        Dense(128, activation='relu'),
        
        # Output Layer
        Dense(num_classes, activation='softmax')
    ])
    
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Main function to load data, train the model, and evaluate
def train_signature_model(base_path):
    # Preprocess the data
    X_train, X_test, y_train, y_test, num_classes, label_encoder = preprocess_data(base_path)

    # Build the CNN model
    model = build_cnn_model(num_classes)
    
    # Train the model
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=EPOCHS, batch_size=BATCH_SIZE)

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {accuracy * 100:.2f}%")

    return model, label_encoder

# Function to predict the signature ID for a given image
def predict_signature(model, image_path, label_encoder):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH))
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    image = image.astype('float32') / 255.0  # Normalize

    # Predict the class
    predictions = model.predict(image)
    predicted_label = np.argmax(predictions)
    
    # Convert label to actual ID using the encoder
    predicted_id = label_encoder.inverse_transform([predicted_label])[0]
    
    return predicted_id

# Path to the directory containing signature folders (each folder named with a unique ID like 1, 2, 3, etc.)
base_directory = 'output_images'

# Train the CNN model
model, label_encoder = train_signature_model(base_directory)

# Predict an example signature
image_path = 'C:\\Users\\Nutech\\Desktop\\Semester 7\\Gen AI\\test code\\genai\\img\\output_images\\4\\aug_0_sign_1.png'
predicted_id = predict_signature(model, image_path, label_encoder)
print(f"Predicted Signature ID: {predicted_id}")


KeyboardInterrupt: 